In [9]:
import cv2
import time
import darknet as dn
import tkinter as tk
from tkinter import Button, Tk, HORIZONTAL
from tkinter.ttk import Progressbar
from PIL import Image
from PIL import ImageTk
from tkinter import filedialog
from tqdm import tqdm
import gc
import csv
tracker_type = "medianflow"
tracker = cv2.TrackerCSRT_create

In [10]:
def dawhtsal(bbox1, bbox2):
    #2 hairtsagnii dawhtsaliig tootsoolno.
    bbox1_x1 = int(bbox1[0])
    bbox1_y1 = int(bbox1[1])
    bbox1_x2 = int(bbox1[0] + bbox1[2])
    bbox1_y2 = int(bbox1[1] + bbox1[3])
    bbox2_x1 = int(bbox2[0])
    bbox2_y1 = int(bbox2[1])
    bbox2_x2 = int(bbox2[0] + bbox2[2])
    bbox2_y2 = int(bbox2[1] + bbox2[3])
    dawhtsal_width = min(bbox1_x2, bbox2_x2) - max(bbox1_x1, bbox2_x1)
    dawhtsal_height = min(bbox1_y2, bbox2_y2) - max(bbox1_y1, bbox2_y1)
    if dawhtsal_width <= 0 or dawhtsal_height <= 0:
        return 0.0
    dawhtsal_area = dawhtsal_width * dawhtsal_height
    bbox1_area = (bbox1_x2 - bbox1_x1) * (bbox1_y2 - bbox1_y1)
    bbox2_area = (bbox2_x2 - bbox2_x1) * (bbox2_y2 - bbox2_y1)
    if bbox1_area < bbox2_area:   
        dawhtsal_huwi = dawhtsal_area / (bbox1_area) 
    else:
        dawhtsal_huwi = dawhtsal_area / (bbox2_area)
    return dawhtsal_huwi

In [11]:
def ogtlol(line1, line2):
    def get_orientation(p, q, r):
        val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
        if val == 0:
            return 0
        return 1 if val > 0 else 2
    def is_on_segment(p, q, r):
        if q[0] <= max(p[0], r[0]) and q[0] >= min(p[0], r[0]) and \
            q[1] <= max(p[1], r[1]) and q[1] >= min(p[1], r[1]):
            return True
        return False
    p1 = line1[0]
    q1 = line1[1]
    p2 = line2[0]
    q2 = line2[1]
    o1 = get_orientation(p1, q1, p2)
    o2 = get_orientation(p1, q1, q2)
    o3 = get_orientation(p2, q2, p1)
    o4 = get_orientation(p2, q2, q1)
    if o1 != o2 and o3 != o4:
        return True
    if o1 == 0 and is_on_segment(p1, p2, q1):
        return True
    if o2 == 0 and is_on_segment(p1, q2, q1):
        return True
    if o3 == 0 and is_on_segment(p2, p1, q2):
        return True
    if o4 == 0 and is_on_segment(p2, q1, q2):
        return True
    return False

def shugam_shalgah(bbox, line):
    x, y, w, h = bbox
    bbox_line1 = [(x, y), (x + w, y)]
    bbox_line2 = [(x + w, y), (x + w, y + h)]
    bbox_line3 = [(x, y), (x, y + h)]
    bbox_line4 = [(x, y + h), (x + w, y + h)]
    if ogtlol(bbox_line1, line) or \
            ogtlol(bbox_line2, line) or \
            ogtlol(bbox_line3, line) or \
            ogtlol(bbox_line4, line):
        return True
    return False

In [12]:
def click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        refPt.append((x,y))

In [13]:
def mashin_detection(bbox):
    results = dn.performDetect("temp.png")
    nbbox = []
    clean = []
    for k in bbox:
        clean.append(k[1])
    for cat, score, bounds in results:
        temp = cat
        if not temp == "car" and not temp == "truck" and not temp == "bus":
            continue
        x, y, w, h = bounds
        bbox1 = (int(x - w / 2),int(y - h / 2),w,h)
        flag=0
        for k,j in enumerate(bbox):
            if j is None:
                continue
            if dawhtsal(bbox1, j[0]) >= 0.5:
                clean[k] = -1
                flag = 1
        if flag == 0 and score>0.8:
            nbbox.append(bbox1)
    for j in range(len(clean)):
        clean[j]+=1
    return nbbox,clean

In [14]:
class Mashin():
    def __init__(self):
        self.root = tk.Tk()
        self.text = []
        self.label = []
        self.progress = Progressbar(self.root, orient = HORIZONTAL, 
              length = 100, mode = 'determinate')
        self.btn = Button(self.root, text="Бичлэг сонгох", command=self.video_unshih)
        self.btn.pack(side="bottom", fill="both", expand="yes", padx="10", pady="10")
    def urdun(self,tooloh):
        zug = {"0":"Хойд","1":"Зүүн","2":"Урд","3":"Баруун"}
        cnt = 0
        for i in tooloh:
            for j in tooloh[i]:
                if cnt < len(self.text):
                    self.text[cnt].set(str(zug[i]+"---> "+zug[j]+" : "+str(tooloh[i][j])))
                else:
                    self.text.append(tk.StringVar())
                    self.text[-1].set(str(zug[i]+"---> "+zug[j]+" : "+str(tooloh[i][j])))
                    self.label.append(tk.Label(self.root, textvariable=self.text[-1]))
                    self.label[-1].pack()
                    self.root.update_idletasks() 
                cnt+=1
    def video_unshih(self):
        global bichleg
        path = filedialog.askopenfilename()
        video = cv2.VideoCapture(path)
        width  = video.get(3)  # float
        height = video.get(4) # float
        if not video.isOpened():
            bichleg = None
            return None
        ok, first_frame = video.read()
        if not ok:
            bichleg = None
            return None
        self.btn.destroy()
        self.btn = Button(self.root, text="Орох,гарах шугам зурах", command=self.line_zurah)
        self.btn.pack(side="bottom", fill="both", expand="yes", padx="10", pady="10")
        bichleg = video,first_frame,width,height
    def line_zurah(self):
        global refPt
        refPt = []
        image = bichleg[1].copy()
        image_list = []
        cv2.namedWindow("Startline")
        cv2.setMouseCallback("Startline", click)
        global start 
        start = []
        cnt=0
        while True:
            refPt = []
            flag_end=False
            while True:
                cv2.imshow("Startline", image)
                key = cv2.waitKey(1) & 0xFF
                if key == ord("r"):
                    if not len(start)==0:
                        start.pop()
                        image_list.pop()
                    if len(image_list)==0:
                        image = bichleg[1].copy()
                    else:
                        image = image_list[-1].copy()
                elif key == ord("c"):
                    flag_end=True
                    break
                if len(refPt) == 2:
                    color = (0, 255, 0)  
                    thickness = 6
                    image = cv2.line(image, refPt[0] , refPt[1] , color, thickness)
                    image_list.append(image.copy())
                    break
            if flag_end:
                break
            start.append(refPt)
        if len(image_list)==0:
            init_image = bichleg[1].copy()
        else:
            init_image = image_list[-1].copy()
        cv2.destroyAllWindows()
        cv2.namedWindow("Endline")
        cv2.setMouseCallback("Endline", click)
        image_list = []
        global end
        end = []
        while True:
            refPt = []
            flag_end=False
            while True:
                cv2.imshow("Endline", image)
                key = cv2.waitKey(1) & 0xFF
                if key == ord("r"):
                    if not len(end)==0:
                        end.pop()
                        image_list.pop()
                    if len(image_list)==0:
                        image = init_image.copy()
                    else:
                        image = image_list[-1].copy()
                elif key == ord("c"):
                    flag_end=True
                    break
                if len(refPt) == 2:
                    color = (0, 0, 255)  
                    thickness = 6
                    image = cv2.line(image, refPt[0] , refPt[1] , color, thickness)
                    image_list.append(image.copy())
                    break
            if flag_end:
                break
            end.append(refPt)
        cv2.destroyAllWindows()
        self.btn.destroy()
        self.btn = Button(self.root, text="Тоолж эхлэх", command=self.mashin_tooloh)
        self.btn.pack(side="bottom", fill="both", expand="yes", padx="10", pady="10")
    def mashin_tooloh(self):
        self.btn.destroy()
        self.loading = tk.StringVar()
        self.loading.set("Эхний фреймээс машин хайж байна...")
        self.temp_label = tk.Label(self.root, textvariable=self.loading)
        self.temp_label.pack()
        tooloh = {"0": {"0": 0, "1": 0, "2": 0, "3": 0}, "1" : {"0": 0, "1": 0, "2": 0, "3": 0}, 
                  "2" : {"0": 0, "1": 0, "2": 0, "3": 0}, "3" : {"0": 0, "1": 0, "2": 0, "3": 0} }
        self.urdun(tooloh)
        self.progress.pack(pady = 1)
        self.root.update_idletasks() 
#         start = [[(323, 604), (303, 501)], [(264, 275), (696, 301)], [(1078, 285), (1271, 427)], [(1273, 570), (1028, 713)]] 
#         end = [[(272, 367), (349, 453)], [(749, 291), (935, 258)], [(1252, 425), (1278, 458)], [(530, 716), (899, 715)]]
        if bichleg:
            video,frame,width,height = bichleg
        else:
            self.loading.set("Бичлэгийг уншиж чадсангүй")
            return 0
        zah=[[(0,0),(width,0)],[(0,0),(0,height)],[(width,0),(width,height)],[(0,height),(width,height)]]
        start_time = time.time()
        bbox=[]
        mashin_trackers=[]
        cv2.imwrite("temp.png", frame)
        results = dn.performDetect("temp.png")
        for cat, score, bounds in results:
            temp = cat
            if not temp == 'car' and not temp == 'truck' and not temp == 'bus':
                continue
            elif score<0.8:
                continue
            x, y, w, h = bounds
            bbox.append([(int(x - w / 2),int(y - h / 2),w,h),0]) 
            mashin_trackers.append([tracker(),0,None,None,0])
        for i in range(len(bbox)):
            ok = mashin_trackers[i][0].init(frame, bbox[i][0])
            for k,j in enumerate(start):
                if shugam_shalgah(bbox[i][0], j):
                    mashin_trackers[i][2]=k
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter('results/Garalt.avi', fourcc, 20.0, (int(width),int(height)))
        print(start,end)
        frame_det = 1
        video_duration = 192
        frame_rate = 25
        total=frame_rate*video_duration
        cnt=0
        self.temp_label.destroy()
        while True:
            ok, frame = video.read()
            cnt+=1
            self.progress['value'] = int((cnt/total)*100)
            self.root.update_idletasks()  
            bbox = list(filter(None, bbox))
            mashin_trackers = list(filter(None, mashin_trackers))
            if not ok:
                out.release()
                break
            if frame_det == 30:
                cv2.imwrite("temp.png", frame)
                nbbox, clean = mashin_detection(bbox)
                for i in range(len(bbox)):
                    if clean[i]>=3:
                        bbox[i] = None
                        mashin_trackers[i] = None
                    else:
                        bbox[i][1]=clean[i]
                for ntemp in nbbox:
                    bbox.append([ntemp,0])
                    mashin_trackers.append([tracker(),0,None,None,0])
                    mashin_trackers[-1][0].init(frame, bbox[-1][0])
                frame_det=0
            for i in range(len(bbox)):
                if mashin_trackers[i] is None:
                    continue
                ok, bbox[i][0] = mashin_trackers[i][0].update(frame)
                mashin_trackers[i][4]+=1
                if mashin_trackers[i][4] == 150 and mashin_trackers[i][2] is None:
                    mashin_trackers[i]=None
                    bbox[i]=None
                    continue
                if mashin_trackers[i][4] == 1000:
                    mashin_trackers[i]=None
                    bbox[i]=None
                    continue
                if mashin_trackers[i][2] is None:
                    for k,j in enumerate(start):
                        if shugam_shalgah(bbox[i][0], j):
                            mashin_trackers[i][2]=k
                            break
                for k,j in enumerate(end):
                    if shugam_shalgah(bbox[i][0], j):
                        mashin_trackers[i][3]=k
                    if not mashin_trackers[i][2] is None and mashin_trackers[i][3] == mashin_trackers[i][2]:
                        mashin_trackers[i][3]=None
                        break
                if mashin_trackers[i][2] and shugam_shalgah(bbox[i][0], start[mashin_trackers[i][2]]):
                    mashin_trackers[i][3]=None
                if ok:
                    p1 = (int(bbox[i][0][0]), int(bbox[i][0][1]))
                    p2 = (int(bbox[i][0][0] + bbox[i][0][2]), int(bbox[i][0][1] + bbox[i][0][3]))
                    cv2.rectangle(frame, p1, p2, (255,0,0), 4, 1)
                else:
                    mashin_trackers[i][1]+=1
                flag2=False
                for k,j in enumerate(zah):
                    if shugam_shalgah(bbox[i][0], j):
                        flag2=True
                        break
                    if not mashin_trackers[i][2] is None and mashin_trackers[i][3] == mashin_trackers[i][2]:
                        flag2=False
                        mashin_trackers[i][3]=None
                        break
                if not mashin_trackers[i][3] is None and not mashin_trackers[i][2] is None:
                    flag2=True
                if flag2:
                    bbox[i]=None
                    s=str(mashin_trackers[i][2])
                    e=str(mashin_trackers[i][3])
                    if not s == "None" and not e == "None":
                        tooloh[s][e]+=1
                        self.urdun(tooloh)
                    mashin_trackers[i] = None
                    continue
                elif mashin_trackers[i][1]>=15:
                    mashin_trackers[i] = None
                    bbox[i]=None
                    continue
            for i in end:
                frame = cv2.line(frame, i[0], i[1], (0, 0, 255), 6)
            for i in start:
                frame = cv2.line(frame, i[0], i[1], (0, 255, 0), 6)
            cv2.imshow("Counting...", frame)
            k = cv2.waitKey(1) & 0xff
            if k == ord("c"):
                break
            out.write(frame)
            frame_det+=1
            gc.collect()
        end_time = time.time()
        cv2.destroyAllWindows()
        print(end_time-start_time)
        global hariu
        hariu = tooloh
        with open('results/garalt.csv', 'w', newline='') as csvfile:
            fieldnames = ['Эхлэл', 'Төгсгөл','Тоо']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            zug = {"0":"Хойд","1":"Зүүн","2":"Урд","3":"Баруун"}
            cnt = 0
            for i in hariu:
                for j in hariu[i]:
                    writer.writerow({'Эхлэл': zug[i], 'Төгсгөл': zug[j], 'Тоо': hariu[i][j] })

In [ ]:
app = Mashin()
app.root.mainloop()

In [8]:
# print(start,end)
# Tsag uur (start ,endline)
# start = [[(1383, 362), (1396, 375)], [(1378, 481), (1043, 471)], [(756, 366), (736, 397)], [(833, 283), (1099, 321)]]
# end = [[(1481, 376), (1483, 408)], [(701, 506), (934, 517)], [(746, 303), (734, 352)], [(1194, 273), (1263, 273)]]
# dulguun nuur (start ,endline)
# start = [[(852, 338), (893, 409)], [(938, 567), (253, 556)], [(78, 384), (3, 428)], [(278, 308), (555, 321)]] 
# end = [[(973, 391), (998, 449)], [(3, 448), (38, 531)], [(45, 330), (0, 368)], [(595, 280), (702, 272)]]
# Sapporo
# start = [[(78, 326), (457, 323)], [(719, 259), (1134, 416)], [(1351, 489), (1056, 660)], [(304, 504), (450, 707)]]
# end = [[(522, 302), (643, 286)], [(1328, 387), (1496, 429)], [(690, 924), (1132, 850)], [(14, 337), (49, 456)]]
# udur2
# start = [[(324, 554), (455, 693)], [(182, 325), (508, 260)], [(878, 184), (1092, 229)], [(1006, 325), (993, 673)]]
# end = [[(92, 452), (240, 717)], [(671, 240), (755, 181)], [(1245, 441), (1258, 580)], [(614, 710), (1080, 638)]]
# udur1
# start = [[(323, 604), (303, 501)], [(264, 275), (696, 301)], [(1078, 285), (1271, 427)], [(1273, 570), (1028, 713)]] 
# end = [[(272, 367), (349, 453)], [(749, 291), (935, 258)], [(1252, 425), (1278, 458)], [(530, 716), (899, 715)]]
# udur3
# start = [[(262, 595), (443, 719)], [(116, 346), (482, 210)], [(774, 81), (1060, 190)], [(1132, 302), (1061, 602)]] 
# end = [[(98, 402), (229, 598)], [(544, 191), (669, 66)], [(1094, 205), (1275, 270)], [(604, 715), (1055, 698)]]
# udur4
# start = [[(1017, 309), (1152, 512)], [(711, 715), (1125, 719)], [(167, 543), (155, 702)], [(151, 318), (462, 296)]] 
# end = [[(1191, 531), (1276, 691)], [(459, 717), (252, 714)], [(73, 352), (75, 527)], [(511, 195), (672, 204)]]
# udur5
# start = [[(483, 665), (442, 500)], [(384, 320), (694, 322)], [(960, 373), (1121, 439)], [(898, 633), (1167, 585)]]
# end = [[(353, 437), (437, 476)], [(801, 325), (910, 323)], [(1201, 454), (1249, 567)], [(643, 715), (875, 680)]]
# shunu1